In [1]:
# Import CANVAS_API_KEY from the .env file
from dotenv import load_dotenv
load_dotenv(verbose=True)

import os
API_KEY = os.getenv("CANVAS_API_KEY")
BASE_URL = os.getenv("CANVAS_BASE_URL")


In [2]:
import pandas as pd
import requests
import json
import numpy as np

In [59]:
def my_request(API_URL):
    headers = {'Authorization': 'Bearer '+ API_KEY}
    r = requests.get(BASE_URL+API_URL, headers=headers)
    return r.json()

In [63]:
r = my_request('/api/v1/courses?enrollment_type=teacher')

response= pd.DataFrame.from_dict(r)
course_ids = response[response['name'].str.contains('AI')]['id']


```
id	name
1116	AI-1
1202	AI-2
1394	AI-3
1574	AI-4
1682	AI-5
```

In [4]:
#let's start by using AI-4 id:1574
course_id =course_ids[4]

### Get student summaries

In [64]:
#Get student_summaries
r = my_request('/api/v1/courses/{}/analytics/student_summaries?per_page=200'.format(course_id))

response= pd.DataFrame.from_dict(r)

In [66]:
tardiness_summary = pd.DataFrame(list(response.tardiness_breakdown))
student_summaries = response.merge(tardiness_summary, left_index=True, right_index=True).drop(['tardiness_breakdown'], axis='columns')
student_summaries.head()

,id,page_views,max_page_views,page_views_level,participations,max_participations,participations_level,missing,late,on_time,floating,total
0,75494,1618,7132,2,51,51,3,30,5,26,32,93
1,74581,1810,7132,2,40,51,3,47,0,30,16,93
2,56876,555,7132,1,42,51,3,32,5,26,30,93
3,74755,1738,7132,2,14,51,1,66,0,11,16,93
4,74578,4609,7132,3,34,51,2,55,1,21,16,93


### Getting Student History

In [114]:
# Get student history
# let's look at id: 75494
course_ids = '1574'
student_id = '75494'
r = my_request('/api/v1/courses/{}/analytics/users/{}/activity?per_page=200'.format(course_ids,student_id))


In [115]:
r['page_views']

{'2020-08-03T20:00:00Z': 19,
 '2020-08-03T23:00:00Z': 10,
 '2020-08-04T14:00:00Z': 13,
 '2020-08-05T12:00:00Z': 10,
 '2020-08-05T13:00:00Z': 9,
 '2020-08-05T14:00:00Z': 54,
 '2020-08-08T23:00:00Z': 2,
 '2020-08-09T21:00:00Z': 2,
 '2020-08-09T22:00:00Z': 23,
 '2020-08-10T13:00:00Z': 16,
 '2020-08-10T16:00:00Z': 4,
 '2020-08-10T18:00:00Z': 3,
 '2020-08-10T19:00:00Z': 4,
 '2020-08-11T16:00:00Z': 14,
 '2020-08-13T11:00:00Z': 5,
 '2020-08-13T12:00:00Z': 16,
 '2020-08-13T13:00:00Z': 12,
 '2020-08-13T18:00:00Z': 6,
 '2020-08-13T19:00:00Z': 5,
 '2020-08-17T14:00:00Z': 10,
 '2020-08-18T13:00:00Z': 2,
 '2020-08-18T15:00:00Z': 26,
 '2020-08-18T17:00:00Z': 6,
 '2020-08-18T18:00:00Z': 7,
 '2020-08-18T19:00:00Z': 14,
 '2020-08-18T20:00:00Z': 7,
 '2020-08-19T13:00:00Z': 25,
 '2020-08-19T14:00:00Z': 25,
 '2020-08-19T15:00:00Z': 5,
 '2020-08-19T18:00:00Z': 17,
 '2020-08-20T13:00:00Z': 15,
 '2020-08-22T15:00:00Z': 58,
 '2020-08-23T22:00:00Z': 2,
 '2020-08-24T13:00:00Z': 18,
 '2020-08-25T11:00:00Z': 1,
 

In [86]:
pd.DataFrame(list(zip(list(r['page_views']),r['page_views'].values())),columns=['date', 'pageviews']).head()

,date,pageviews
0,2020-08-03T20:00:00Z,19
1,2020-08-03T23:00:00Z,10
2,2020-08-04T14:00:00Z,13
3,2020-08-05T12:00:00Z,10
4,2020-08-05T13:00:00Z,9


In [87]:
pd.DataFrame.from_dict(r['participations']).head()

,created_at,url
0,2020-08-05T14:32:10Z,https://student.emeritus.org/courses/1574/quiz...
1,2020-08-05T14:32:34Z,https://student.emeritus.org/courses/1574/quiz...
2,2020-08-05T14:49:26Z,https://student.emeritus.org/api/v1/courses/15...
3,2020-08-10T18:23:03Z,https://student.emeritus.org/api/v1/courses/15...
4,2020-08-13T12:46:09Z,https://student.emeritus.org/api/v1/courses/15...


### Get student outcomes

In [119]:
#get sections
course_id = 1574
                                      
r = my_request('/api/v1/courses/{}/sections?per_page=200'.format(course_id))

In [121]:
pd.DataFrame.from_dict(r)[['id','name']]

,id,name
0,4341,Berkeley Haas Artificial Intelligence
1,4606,Deferment
2,4607,Observers
3,4608,Section A
4,4609,Section B
5,4610,Section C
6,4604,Support
7,4605,Withdrawal


In [145]:
#get assignment id (contains all sections)
course_id = '1574'
                                      
r = my_request('/api/v1/courses/{}/assignments?per_page=200'.format(course_id))

In [146]:
pd.DataFrame.from_dict(r)['discussion_topic'].iloc[0]

{'id': 93596,
 'title': 'Discussion 3.1 - Penalizing Mistakes - Section A',
 'last_reply_at': '2020-08-28T15:53:31Z',
 'created_at': '2020-07-28T13:36:27Z',
 'delayed_post_at': None,
 'posted_at': '2020-08-21T11:32:52Z',
 'assignment_id': 63392,
 'root_topic_id': None,
 'position': None,
 'podcast_has_student_posts': False,
 'discussion_type': 'threaded',
 'lock_at': None,
 'allow_rating': True,
 'only_graders_can_rate': False,
 'sort_by_rating': False,
 'is_section_specific': False,
 'user_name': None,
 'discussion_subentry_count': 18,
 'permissions': {'attach': True,
  'update': True,
  'reply': True,
  'delete': True},
 'require_initial_post': None,
 'user_can_see_posts': True,
 'podcast_url': None,
 'read_state': 'unread',
 'unread_count': 18,
 'subscribed': False,
 'attachments': [],
 'published': True,
 'can_unpublish': False,
 'locked': False,
 'can_lock': True,
 'comments_disabled': False,
 'author': {},
 'html_url': 'https://student.emeritus.org/courses/1574/discussion_topics/

In [142]:
#get submissions per section
section_id = '4608'
assignment_id = '63393'
                                      
r = my_request('/api/v1/sections/{}/assignments/{}/submissions?per_page=200'.format(section_id,assignment_id))

In [144]:
pd.DataFrame.from_dict(r).head()

,id,body,url,grade,score,submitted_at,assignment_id,user_id,submission_type,workflow_state,...,extra_attempts,posted_at,late,missing,seconds_late,entered_grade,entered_score,preview_url,discussion_entries,attachments
0,6175155,None,None,complete,0.0,2020-08-24T01:25:08Z,63393,62834,discussion_topic,graded,...,None,2020-09-25T09:38:04Z,False,False,0,complete,0.0,https://student.emeritus.org/courses/1574/assi...,"[{'id': 897084, 'user_id': 62834, 'parent_id':...",NaN
1,6189583,None,None,complete,0.0,2020-08-25T18:58:34Z,63393,63186,discussion_topic,graded,...,None,2020-09-25T09:38:05Z,False,False,0,complete,0.0,https://student.emeritus.org/courses/1574/assi...,"[{'id': 899915, 'user_id': 63186, 'parent_id':...",NaN
2,6159137,None,None,complete,0.0,2020-08-28T03:12:08Z,63393,70835,discussion_topic,graded,...,None,2020-09-25T09:38:00Z,False,False,0,complete,0.0,https://student.emeritus.org/courses/1574/assi...,"[{'id': 904459, 'user_id': 70835, 'parent_id':...",NaN
3,6159138,None,None,complete,0.0,2020-08-27T14:27:57Z,63393,70836,discussion_topic,graded,...,None,2020-09-25T09:37:58Z,False,False,0,complete,0.0,https://student.emeritus.org/courses/1574/assi...,"[{'id': 903555, 'user_id': 70836, 'parent_id':...",NaN
4,6159141,None,None,complete,0.0,2020-08-28T03:20:46Z,63393,70839,discussion_topic,graded,...,None,2020-09-25T09:38:07Z,False,False,0,complete,0.0,https://student.emeritus.org/courses/1574/assi...,"[{'id': 904466, 'user_id': 70839, 'parent_id':...",NaN


In [147]:
#get list of assignments per student
course_id = '1574'
student_id = '74581'

r = my_request('/api/v1/courses/{}/students/submissions?student_ids[]={}&per_page=200'.format(course_id,student_id))

In [152]:
pd.DataFrame.from_dict(r)[['grade','assignment_id','late','missing','seconds_late']]

,grade,assignment_id,late,missing,seconds_late
0,complete,63387,False,False,0
1,complete,63388,False,False,0
2,complete,63389,False,False,0
3,complete,63390,False,False,0
4,complete,63391,False,False,0
5,complete,63392,False,False,0
6,complete,63393,False,False,0
7,complete,63396,False,False,0
8,complete,63676,False,False,0
9,complete,63679,False,False,0


In [78]:
# get capstone project success rate assignment_id=65600
course_id = 1574
assignment_id = 65600 #to-do get list of assignments
r = my_request('/api/v1/courses/{}/assignments/{}/submissions'.format(course_id,assignment_id))

In [91]:
response= pd.DataFrame.from_dict(r)
response[response['id']==63393]


,id,description,due_at,unlock_at,lock_at,points_possible,grading_type,assignment_group_id,grading_standard_id,created_at,...,needs_grading_count,discussion_topic,published,unpublishable,only_visible_to_overrides,locked_for_user,submissions_download_url,post_manually,anonymize_students,require_lockdown_browser
1,63393,"<link rel=""stylesheet"" href=""https://instructu...",None,2020-08-21T00:00:00Z,None,0.0,pass_fail,2989,None,2020-07-28T13:36:27Z,...,0,"{'id': 93597, 'title': 'Assignment 3.1 - Neura...",True,False,True,False,https://student.emeritus.org/courses/1574/assi...,False,False,False


In [94]:
response[response['id']==63373]

,id,description,due_at,unlock_at,lock_at,points_possible,grading_type,assignment_group_id,grading_standard_id,created_at,...,needs_grading_count,discussion_topic,published,unpublishable,only_visible_to_overrides,locked_for_user,submissions_download_url,post_manually,anonymize_students,require_lockdown_browser
38,63373,"<link rel=""stylesheet"" href=""https://instructu...",None,2020-08-21T00:00:00Z,None,0.0,pass_fail,2989,None,2020-07-28T13:36:23Z,...,0,"{'id': 93574, 'title': 'Assignment 3.1 - Neura...",True,False,True,False,https://student.emeritus.org/courses/1574/assi...,False,False,False


In [95]:
response[response['id']==63374]

,id,description,due_at,unlock_at,lock_at,points_possible,grading_type,assignment_group_id,grading_standard_id,created_at,...,needs_grading_count,discussion_topic,published,unpublishable,only_visible_to_overrides,locked_for_user,submissions_download_url,post_manually,anonymize_students,require_lockdown_browser
39,63374,"<link rel=""stylesheet"" href=""https://instructu...",None,2020-08-21T00:00:00Z,None,0.0,pass_fail,2989,None,2020-07-28T13:36:23Z,...,0,"{'id': 93575, 'title': 'Assignment 3.1 - Neura...",True,False,True,False,https://student.emeritus.org/courses/1574/assi...,False,False,False


In [98]:
response['assignment_group_id'].unique()

array([2989])